In [4]:
import os
import numpy as np
import pandas as pd
from itertools import combinations, permutations

def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string


def make_table(path):
    # read file names(id)
    list_of_files = os.listdir(path) # len=560
    index_list = []
    for file_name in list_of_files:
        # index_list.append(int(file_name.removesuffix('.txt')))  # for python 3.9+
        index_list.append(int(remove_suffix(file_name, '.txt')))

    # read file content
    titles = []
    texts = []
    notices = []
    for file in list_of_files:
        f = open("{}/{}".format(path, file), "r")
        temp = f.readlines()
        titles.append(temp[0])
        texts.append(temp[1])
        try:
            notices.append(temp[2])
        except:
            notices.append('')
        f.close()

    # make general table
    df_dirty = pd.DataFrame(
        {'ID': index_list,
         'title': titles,
         'text': texts,
         'notice': notices
        })
    return df_dirty



def make_data(dataType, table):
    
    # make text_pairs, data scope can be changed, I only include "title" here
    # number of text_pairs = Pn取2, train:n=560; test:n=421

    combs = permutations(table[['ID', 'title']].values, r=2)
    combs = pd.DataFrame([[s for s in comb] for comb in combs], columns=['text_a', 'text_b'])
    # combs[['id_text_b', 'text_b']] = combs['text_b'].to_list()
    temp_l = pd.DataFrame(combs['text_a'].to_list(), columns = ['id_text_a', 'text_a'])
    temp_r = pd.DataFrame(combs['text_b'].to_list(), columns = ['id_text_b', 'text_b'])
    combs = pd.concat([temp_l, temp_r], axis=1)
    

    if dataType=='train':
        # make label set
        labels = pd.read_csv('TrainLabel.csv')
        pairs = set()
        for _, row in labels.iterrows():
            temp = frozenset([row['Test'], row['Reference']])
            if temp not in pairs:
                pairs.add(temp)

        # mark label on text_pairs
        combs['label'] = 'unlike'
        for ind, row in combs.iterrows():
            if {row['id_text_a'], row['id_text_b']} in pairs:
                combs.at[ind, 'label'] = 'like'

    
    # combs.to_csv("{}.tsv".format(dataType), sep="\t", index=False)            
    return combs


In [10]:
table_train = make_table("./dataTrainComplete")
data_train = make_data('train', table_train)
data_train = data_train[['text_a', 'text_b', 'label']]
data_train.to_csv("train.tsv", sep="\t", index=False)


table_test = make_table("./dataPublicComplete")
data_test = make_data('test', table_test)
data_test = data_test[['text_a', 'text_b', 'id_text_a', 'id_text_b']]
data_test.to_csv("test.tsv", sep="\t", index=False)



In [6]:
table_train.head(10)

,ID,title,text,notice
0,1301,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,一、近期時有細雨，加上早晚溫度低、露水量多，有利香蕉葉部病菌之感染及漫延，造成葉片嚴重發病及...,噴藥方式及藥劑配方如下列：(一)、肩背式微粒動力噴霧機：每公頃80%鋅錳乃浦可濕性粉劑2.3...
1,924,二期作水稻已陸續完成插秧，請留意防治水稻水象鼻蟲\n,二期作水稻已陸續插秧，請留意防治水稻水象鼻蟲行政院農業委員會桃園區農業改良場表示，目前二期作...,\n
2,595,鹿野鄉局部地區水稻胡麻葉枯病發生嚴重，臺東場籲請該地區農民及時防治\n,臺東區農業改良場（以下簡稱臺東場）表示，該場於進行水稻病蟲害監測調查及相關試驗時，發現鹿野鄉...,防治方法及應注意事項：一、施行稻種消毒；勿選用罹病老熟秧苗，以免傳播病原。二、發病嚴重田，於...
3,438,臺南區農改場籲請農友注意水稻白葉枯病、稻熱病、紋枯病與葉鞘腐敗病之發生與防治。臺南區農改場、...,水稻已至白葉枯病發病高峰期，田間也陸續發生白葉枯病，且稻熱病與紋枯病持續發展，臺南區農改場籲...,農友應注意葉稻熱病、紋枯病與白葉枯病在田間是否發生，倘若發生應立即用藥防治。葉鞘腐敗病為水稻...
4,344,早晚溫差大濕度高，臺南場籲請農友加強水稻稻熱病與白葉枯病防治。\n,近日台南地區早晚溫差大濕度高，罹患水稻白葉枯病與葉稻熱病有增加之趨勢，二期水稻也進入孕穗至抽...,水稻白葉枯病的病原細菌可由水孔或傷口侵入，常在葉片邊緣形成波浪狀病斑，隨著葉脈往下蔓延，在葉...
5,746,加強杜絕洋香瓜病毒病蔓延\n,雲嘉南地區洋香瓜目前正值生產季節，由於降雨量少，小型昆蟲包括蚜蟲、薊馬、銀葉粉蝨密度持續增加...,\n
6,1382,稻熱病蠢蠢欲動，高雄區農業改良場籲請農友及早防範\n,行政院農業委員會高雄區農業改良場（以下簡稱高雄場）表示，目前高屏地區日夜溫差較大，已逐漸出現...,稻熱病的防治方法，應依據植物保護手冊規定，選用適當藥劑進行施用防治，例如15%加普胺水懸劑、...
7,453,目前正值稻熱病流行高峰期，花蓮區農業改良場籲請農友注意加強防治\n,一、發生地區及防治適期：花蓮縣水稻栽培區大部份已有葉稻熱病發生，應注意田間發病情形並及時施藥...,\n
8,984,台中區農業改良場發佈水稻白葉枯病發生警報，呼籲轄區內農友發動共同防治工作，以免農作物遭受病蟲...,糧食作物病蟲害發生警報中華民國90年9月24日第肆號臺中區農業改良場彰化縣大村鄉松槐路370...,\n
9,1271,香蕉所籲請蕉農務必做好開花蕉株果房疏果、整梳及套袋工作，以提高蕉果品質。\n,一、蕉株生育期間適時防治病蟲害，可減少蕉葉感染病蟲害機會，以維持蕉果正常生長。二、疏果整梳及...,三、疏果、整梳方法：(一)、香蕉終花前分二~三次摘除蕉花，以減少蕉乳污染，並同時進行疏果、整...


In [11]:
data_train

,text_a,text_b,label
0,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,二期作水稻已陸續完成插秧，請留意防治水稻水象鼻蟲\n,unlike
1,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,鹿野鄉局部地區水稻胡麻葉枯病發生嚴重，臺東場籲請該地區農民及時防治\n,unlike
2,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,臺南區農改場籲請農友注意水稻白葉枯病、稻熱病、紋枯病與葉鞘腐敗病之發生與防治。臺南區農改場、...,unlike
3,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,早晚溫差大濕度高，臺南場籲請農友加強水稻稻熱病與白葉枯病防治。\n,unlike
4,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,加強杜絕洋香瓜病毒病蔓延\n,unlike
...,...,...,...
311917,臺東縣水稻一期作請注意防治褐飛蝨、斑飛蝨及穗稻熱病，以確保稻穀產量與品質\n,水稻害蟲密度逐漸升高，高雄場籲請農友注意防範。\n,unlike
311918,臺東縣水稻一期作請注意防治褐飛蝨、斑飛蝨及穗稻熱病，以確保稻穀產量與品質\n,氣溫日降，臺南農改場籲加強防治番茄及馬鈴薯晚疫病以減少損害\n,unlike
311919,臺東縣水稻一期作請注意防治褐飛蝨、斑飛蝨及穗稻熱病，以確保稻穀產量與品質\n,發布本(104)年度第二期作水稻白葉枯病發生預報。桃園區農改場、防檢局及田邊好幫手關心您。\n,unlike
311920,臺東縣水稻一期作請注意防治褐飛蝨、斑飛蝨及穗稻熱病，以確保稻穀產量與品質\n,水稻分蘗盛期籲請農友做好水稻紋枯病及白葉枯病防治工作\n,unlike


In [12]:
data_test

,text_a,text_b,id_text_a,id_text_b
0,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,香蕉所籲請蕉農加強香蕉黑星病及葉斑病防治作業，以確保香蕉良好之品質及產量\n,128,1285
1,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,水稻秧苗病害管理\n,128,927
2,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,春季陰晴不定，請農友儘早預防稻熱病。臺東農改場、防檢局及田邊好幫手關心您。\n,128,294
3,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,時值外銷季節及夏蕉抽穗期，籲請蕉農做好開花蕉株果房疏果、整梳、套袋及蕉園灌溉、排水工作，俾確...,128,1351
4,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,請宣導蕉農做好香蕉黑星病防治作業，以確保香蕉良好之品質及產量。\n,128,1216
...,...,...,...,...
176815,全民一起來監測紅火蟻\n,請加強宣導 貴單位轄區香蕉黑星病防治工作，以確保香蕉良好之品質及產量。\n,1004,1213
176816,全民一起來監測紅火蟻\n,台南場表示柑橘類進入盛花期，請農友注意蚜蟲及薊馬危害，加強防範措施，避免果實受損。\n,1004,363
176817,全民一起來監測紅火蟻\n,臺南農改場表示芒果進入幼果期，炭疽病易發生，防檢局籲請農友多加注意，及時進行防治。\n,1004,367
176818,全民一起來監測紅火蟻\n,水稻瘤野螟已開始發生台南農改場提醒農友注意防治\n,1004,666


In [7]:
from sklearn.utils import shuffle

temp_like = data_train.loc[data_train['label']=='like'][['text_a', 'text_b', 'label']].iloc[50:]
temp_unlike = data_train.loc[data_train['label']=='unlike'][['text_a', 'text_b', 'label']].iloc[50:]


shuffle(pd.concat([temp_like, temp_unlike], axis=0)).to_csv("test.tsv", sep='\t', index=False)
# to_csv("{}.tsv".format(dataType), sep="\t", index=False)

In [3]:
# make observe


temp_like = data_train.loc[data_train['label']=='like'][['text_a', 'text_b', 'label']].iloc[:100]
temp_unlike = data_train.loc[data_train['label']=='unlike'][['text_a', 'text_b', 'label']].iloc[:100]
pd.concat([temp_like, temp_unlike], axis=0).to_csv("./observe/mix.csv", index=False)

In [11]:
import re

rep = {"condition1": "", "condition2": "text"} # define desired replacements here


rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], "(condition1) and --condition2--")
text

'() and --text--'

In [46]:
# fin the most used name

# dic_pest=[]
# for kind in list_pest:
#     kind_dict = dict.fromkeys(kind, 0)
#     dic_pest.append(kind_dict)

    
for ele in table_train['title']:
    
    ele.count('')

In [36]:
dic_pest

[{'太平洋臀紋粉介殼蟲': 0},
 {'番茄黃化捲葉病毒': 0, '番茄捲葉病': 0},
 {'番茄斑點萎凋病毒': 0},
 {'瓜類退綠黃化病毒': 0},
 {'霍香薊黃脈病毒': 0},
 {'褪綠斑駁病毒病': 0},
 {'臺灣本土黑螞蟻': 0, '琉璃蟻': 0},
 {'細菌性黑斑病': 0, '黑斑病': 0, '鬼頭病': 0, '輪斑病': 0},
 {'水稻縞葉枯病': 0},
 {'水稻水象鼻蟲': 0},
 {'番茄嵌紋病毒': 0},
 {'胡瓜嵌紋病毒': 0},
 {'馬鈴薯病毒Y': 0},
 {'細菌性條斑病': 0},
 {'二點小綠葉蟬': 0},
 {'細菌性斑點病': 0},
 {'細菌性角斑病': 0, '臭葉': 0},
 {'柑橘窄胸天牛': 0},
 {'南瓜捲葉病毒': 0},
 {'白輪盾介殼蟲': 0},
 {'地中海果實蠅': 0},
 {'路易氏始葉螨': 0},
 {'斜紋夜盜蟲': 0, '斜紋夜蛾': 0, '行軍蟲': 0, '黑肚蟲': 0, '夜盜蟲': 0, '黑蟲': 0},
 {'稻縱捲葉蟲': 0, '縱捲葉蟲': 0, '瘤野螟': 0, '捲葉蟲': 0, '葉尾蟲': 0},
 {'東方果實蠅': 0, '蜂仔': 0},
 {'草狀矮化病': 0},
 {'皺縮矮化病': 0},
 {'壯鋏普癭蚋': 0},
 {'咖啡木蠹蛾': 0},
 {'台灣黃毒蛾': 0},
 {'小白紋毒蛾': 0},
 {'草地貪夜蛾': 0, '秋行軍蟲': 0},
 {'胡麻葉枯病': 0},
 {'花姬捲葉蛾': 0},
 {'神澤氏葉螨': 0},
 {'葉鞘腐敗病': 0},
 {'柑橘赤葉螨': 0},
 {'黑條萎縮病': 0},
 {'秀粉介殼蟲': 0},
 {'黑點根腐病': 0},
 {'番茄斑潛蠅': 0},
 {'番茄病毒病': 0},
 {'線蟲白尖病': 0},
 {'亞洲潛葉蠅': 0},
 {'番茄潛旋蛾': 0},
 {'臺灣花薊馬': 0},
 {'立枯絲核菌': 0},
 {'假莖象鼻蟲': 0},
 {'球莖象鼻蟲': 0},
 {'疣胸琉璃蟻': 0},
 {'甜菜夜蛾': 0, '蔥仔管蟲': 0, '管仔蟲': 0},
 {'白背飛蝨':

In [39]:
list_chem

[['純白鏈黴菌素700PCU/g水溶性粉劑', '純白鏈黴菌素水溶性粉劑', '純白鏈黴菌素'],
 ['液化澱粉芽孢桿菌YCMA1可溼性粉劑', '液化澱粉芽孢桿菌'],
 ['施亞磷酸溶液；亞磷酸；氫氧化鉀',
  '亞磷酸；氫氧化鉀；溶氫氧化鉀',
  '亞磷酸氫氧化鉀中和之溶液',
  '亞磷酸氫氧化鉀中和之溶液',
  '亞磷酸混和氫氧化鉀',
  '亞磷酸+氫氧化鉀',
  '亞磷酸混氫氧化鉀'],
 ['克熱淨(烷苯磺酸鹽)可濕性粉劑', '克熱淨(烷苯磺酸鹽)'],
 ['加保利可濕性粉劑可濕性粉劑', '加保利可溼性粉劑水懸劑', '加保利可濕性粉劑稀釋', '加保利可濕性粉劑', '加保利'],
 ['克熱淨可濕性粉劑1500倍', '克熱淨可濕性粉劑', '克熱淨溶液', '克熱淨'],
 ['可濕性硫磺水分散性粒劑稀釋',
  '可濕性硫磺可濕性粉劑稀釋3',
  '可濕性硫磺可濕性粉劑稀釋',
  '可濕性硫磺水分散性粒劑',
  '可濕性硫磺粉系列葯劑',
  '可濕性硫磺粉劑',
  '可濕性硫磺'],
 ['鹼性氯氧化銅水份散性粒劑', '鹼性氯氧化銅可濕性粉劑', '鹼性氯氧化銅'],
 ['達滅芬可濕性粉劑或水懸劑', '達滅芬可濕性粉劑', '達滅芬'],
 ['鋅錳右滅達樂水分散性粒劑', '鋅錳右滅達樂'],
 ['益洛寧可濕性粉劑的為害。', '益洛寧可濕性粉劑', '益洛寧'],
 ['可濕性硫黃水分散性粒劑', '可濕性硫黃可濕性粉劑', '可濕性硫黃粉劑', '可濕性硫黃'],
 ['四氯異苯腈水分散性粒劑', '四氯異苯腈可濕性粉劑', '四氯異苯睛可濕性粉劑', '四氯異苯晴可濕性粉劑', '四氯異苯腈'],
 ['賽普護汰寧水分散性粒劑', '賽普護汰寧'],
 ['賽普待克利水分散性粒劑', '賽普待克利'],
 ['賽座滅水懸劑2000倍', '賽座滅水懸劑', '賽座滅'],
 ['鋅錳座賽胺水分散性粒劑', '鋅錳座賽胺'],
 ['嘉賜貝芬混合可濕性粉劑', '嘉賜貝芬'],
 ['嘉磷塞(異丙胺鹽)溶液', '嘉磷塞異丙胺鹽'],
 ['甲基多保淨可濕性粉劑', '甲基多保淨水懸劑', '甲基多保淨軟膏劑', '甲基多保淨'],
 ['嘉賜三賽唑可濕性粉劑', '嘉賜三塞唑可濕性粉劑', '加賜三

In [40]:
list_crop

[['玉荷包荔枝', '荔枝'],
 ['橙蜜香番茄', '橙果小番茄', '小果番茄', '番茄'],
 ['晚崙西亞橙', '晚崙西亞', '丁香'],
 ['禾本科雜草'],
 ['小花蔓澤蘭', '薇甘菊'],
 ['心型薇甘菊', '蔓澤蘭'],
 ['麻豆文旦', '文旦柚', '文旦'],
 ['十字花科'],
 ['台灣欒樹', '欒樹'],
 ['草莓苗期', '草莓'],
 ['鳳梨釋迦', '釋迦'],
 ['明尼吉柚'],
 ['景觀樹木'],
 ['綠肥作物'],
 ['可可椰子'],
 ['結球白菜'],
 ['蔓性作物'],
 ['園藝作物'],
 ['青江白菜'],
 ['包心白菜'],
 ['包心芥菜'],
 ['觀葉植物'],
 ['結球菜類'],
 ['飲料作物'],
 ['青梗白菜'],
 ['天南星科'],
 ['盤固拉草'],
 ['觀賞植物'],
 ['無患子科'],
 ['龍鬚菜'],
 ['洋香瓜'],
 ['落花生', '花生'],
 ['中國梨', '高接梨', '梨'],
 ['番石榴', '芭樂'],
 ['印度棗'],
 ['馬鈴薯'],
 ['紅龍果'],
 ['瓜果類'],
 ['豆菜類'],
 ['洋桔梗'],
 ['百香果'],
 ['番荔枝'],
 ['茂谷柑'],
 ['水蜜桃'],
 ['金針菜', '金針'],
 ['西施柚'],
 ['蔬果類'],
 ['葡萄柚'],
 ['葉菜類'],
 ['葫蘆科'],
 ['芸香科'],
 ['花椰菜'],
 ['聖誕紅'],
 ['薰衣草'],
 ['多花菊'],
 ['唐菖蒲', '唐菖浦'],
 ['花胡瓜'],
 ['九層塔'],
 ['非洲菊'],
 ['佛手瓜'],
 ['七里香', '月桔'],
 ['烏柑仔'],
 ['芥藍菜'],
 ['火鶴花'],
 ['蝴蝶蘭'],
 ['茭白筍', '茭白荀', '筊白筍'],
 ['甜蜜桃'],
 ['小白菜'],
 ['青花菜'],
 ['花菜類'],
 ['草花類'],
 ['盆花類'],
 ['空心菜'],
 ['向日葵'],
 ['旋花科'],
 ['睡蓮科'],
 ['茄果類'],
 ['瓜菜類'],
 ['曼陀羅'],
 ['狼尾草'],
 ['果菜類'],
 ['洛神葵'],


In [2]:
import pandas as pd

labels = pd.read_csv('TrainLabel.csv')